In [11]:
import numpy as np

def index_to_game(num,m):
    # returns an int p (denoting number of vertical sticks) and a m-tuple (denoting number of horizontal sticks)
    # CAUTION: p is not the index of the right-most vertical. Offset by 1, it counts the number of verticals.

    # In the following, we assume such an order of games: First, we order by p. Within each "p group", we order as if they are numbers.

    # first we find out the corresponding p group
    p, total=m+1, 3**(m+1)-2**m

    # special case on p=m+1: need to skip the calculations. CAUTION: game is indexed starting at 0.
    if num + 1 + 2**m > total:
        total-=2**m
    else:
        total-=2**m
        p-=1
        while num + 1 + (2**p)*(3**(m-p))<= total:
            total-=(2**p)*(3**(m-p))
            p-=1

        total-=(2**p)*(3**(m-p))

    # at this point, total < num
    
    # figure out the the m-tuple. 
    if p<m+1:
        front_decimal=(num-total)//(3**(m-p))
        back_decimal=(num-total)%(3**(m-p))
    else:
        front_decimal=(num-total)
        back_decimal=0

    front_binary=int(np.base_repr(front_decimal,base=2))
    back_ternary=int(np.base_repr(back_decimal,base=3))

    m_list=np.zeros(m,dtype=int)
    for i in range(min(p,m)-1,-1,-1): #from p-1 to 0
        m_list[i]=front_binary%10

        if p==m+1 or i<min(p,m)-1: # not the special block right after right-most vertical
            m_list[i]*=2
        
        front_binary=front_binary//10

    for i in range(m-1,min(p,m)-1,-1): #from p-1 to 0
        m_list[i]=back_ternary%10
        
        back_ternary=back_ternary//10
    
    return p,m_list

def is_sub_game(G,H):
    return (G[0]>=H[0] and (G[1]>=H[1]).all())

def coef(G,H): 
    # G is a legal matchstick game on nx1, H is a sub game of G
    # Counts the number of matchstick games on nx1x1, with bottom face G and top face H
    G_list, H_list=G[1],H[1]
    big_mark, top_mark=G_list.size,G_list.size

    if H[0]<G[0]: # naked vertical, can't keep traveling top line
        top_mark= H[0]-1

    #CAUTION: here we're only accounting for horizontal lines
    for i in range(G_list.size):
        if G_list[i]>H_list[i]:
            top_mark=min(i,top_mark)
            if H_list[i]==0:
                big_mark=i
                break
    
    total=0

    if big_mark <= H[0]-1 or H[0]==0: # count the empty blocks. Note that we can always exit at big mark, hence the +1. 
        total+= np.count_nonzero(H_list[:big_mark]==0)+2
        if H[0]==0 and G[0]==0: # no naked vertical, can still travel top line
            for i in range(0,top_mark+1):
                if i==H_list.size:
                    total+=1 # all the way to the end, one way down
                elif H_list[i]<2: # a gap on top
                    total+=np.count_nonzero(H_list[i:big_mark]==0)+1

    else:
        total+=np.count_nonzero(H_list[:H[0]-1]==0)+1 # before p, blocks
        
        for i in range(max(H[0]-1,0),top_mark+1):
            if i==H_list.size:
                total+=1 # all the way to the end, one way down
            elif H_list[i]<2: # a gap on top
                total+=np.count_nonzero(H_list[i:big_mark]==0)+1
            
    return total
    

In [12]:
def make_matrix(m):
    # there are 3^(m+1)-2^m games on mx1x1

    matrix = np.zeros((3**(m+1)-2**m, 3**(m+1)-2**m),dtype=int)
    
    for i in range(3**(m+1)-2**m): # iterating rows

        for j in range(i+1): 
        # iterating each row. Note that due to our ordering, entries left to the diagonal (i,i) are 0s.

            if is_sub_game(index_to_game(i,m),index_to_game(j,m)):
                matrix[i][j]=coef(index_to_game(i,m),index_to_game(j,m))

    return matrix

# A potentially more efficient algorithm: instead of iterating over the entire row corresponding to game G, 
# we enumerate all subgames of G and just update the corresponding entries on that row. But that requires a good enumeration algorithm.

In [13]:
make_matrix(1)

array([[6, 0, 0, 0, 0, 0, 0],
       [3, 4, 0, 0, 0, 0, 0],
       [3, 3, 3, 0, 0, 0, 0],
       [3, 0, 0, 4, 0, 0, 0],
       [2, 2, 0, 2, 3, 0, 0],
       [3, 0, 0, 3, 0, 3, 0],
       [2, 2, 2, 2, 2, 2, 2]])

In [15]:
import sympy as sp
from sympy.abc import i, k, m, n, x
A=sp.Matrix(make_matrix(1).tolist())
((A**n)*sp.Matrix(7*[1])).dot(sp.Matrix(7*[1]))

2**n/2 + 3**n - 12*4**n + 35*6**n/2